# Azure Cosmos DB - Connect using Key Vault

In an earlier lesson ([Key Vault-backed Secret Scopes]($./08-Key-Vault-backed-secret-scopes)), you created a Key Vault-backed secret scope for Azure Databricks, and securely stored your Azure Cosmos DB key within.

In this lesson, you will use the Azure Cosmos DB secrets that are securely stored within the Key Vault-backed secret scope to connect to your Azure Cosmos DB instance. Next, you will use the DataFrame API to execute SQL queries and control the parallelism of reads through the JDBC interface.

If you are running in an Azure Databricks environment that is already pre-configured with the libraries you need, you can skip to the next cell. To use this notebook in your own Databricks environment, you will need to create libraries, using the [Create Library](https://docs.azuredatabricks.net/user-guide/libraries.html) interface in Azure Databricks. Follow the steps below to attach the `azure-eventhubs-spark` library to your cluster:

1. In the left-hand navigation menu of your Databricks workspace, select **Workspace**, select the down chevron next to **Shared**, and then select **Create** and **Library**.

  ![Create Databricks Library](https://databricksdemostore.blob.core.windows.net/images/08/03/databricks-create-library.png 'Create Databricks Library')

2. On the New Library screen, do the following:

  - **Source**: Select Maven Coordinate.
  - **Coordinate**: Enter "azure-cosmosdb-spark", and then select **com.microsoft.azure:azure-cosmosdb-spark_2.3.0_2.11:1.2.7** (or later version).
  - Select **Create Library**.
  
  ![Databricks new Maven library](https://databricksdemostore.blob.core.windows.net/images/04-MDW/cosmos-db-create-library.png 'Databricks new Maven library')

3. On the library page that is displayed, check the **Attach** checkbox next to the name of your cluster to run the library on that cluster.

  ![Databricks attach library](https://databricksdemostore.blob.core.windows.net/images/04-MDW/cosmos-db-attach-library.png 'Databricks attach library')

Once complete, return to this notebook to continue with the lesson.

### Getting Started

Run the following cell to configure our "classroom."

In [3]:
%run "./Includes/Classroom-Setup"

## Access Key Vault secrets and configure the Cosmos DB connector

In a previous lesson, you created two secrets in Key Vault for your Azure Cosmos DB instance: **cosmos-uri** and **cosmos-key**. These two values will be used to configure the Cosmos DB connector. Let's start out by retrieving those values and storing them in new variables.

In [5]:
uri = dbutils.secrets.get(scope = "key-vault-secrets", key = "cosmos-uri")
key = dbutils.secrets.get(scope = "key-vault-secrets", key = "cosmos-key")

In order to query Cosmos DB, you need to first create a configuration object that contains the configuration information. 

If you are curious, read the [configuration reference](https://github.com/Azure/azure-cosmosdb-spark/wiki/Configuration-references) for details on all of the options. 

The core items you need to provide are:

**Endpoint**: Your Cosmos DB url (i.e. https://youraccount.documents.azure.com:443/)

**Masterkey**: The primary or secondary key string for you Cosmos DB account

**Database**: The name of the database

**Collection**: The name of the collection that you wish to query

In [7]:
readConfig = {
"Endpoint" : uri, # from Key Vault
"Masterkey" : key, # from Key Vault
"Database" : "demos",
"Collection" : "documents",
"preferredRegions" : "Central US", # or whichver region you added Cosmos DB to
"SamplingRatio" : "1.0",
"schema_samplesize" : "1000",
"query_pagesize" : "2147483647",
}

## Load parquet data into DataFrames and normalize

We want to normalize New York and Boston data sets and join them into a new DataFrame. This way, we can write the joined data into our Cosmos DB collection so that data can be accessed by external services and applications.  

Start by creating a DataFrame of the data from New York and then Boston:

| City          | Table Name              | Path to DBFS file
| ------------- | ----------------------- | -----------------
| **New York**  | `CrimeDataNewYork`      | `dbfs:/mnt/training/crime-data-2016/Crime-Data-New-York-2016.parquet`
| **Boston**    | `CrimeDataBoston`       | `dbfs:/mnt/training/crime-data-2016/Crime-Data-Boston-2016.parquet`

Then, normalize the data structure of each table so all the columns (and their values) line up with each other.

In the case of New York and Boston, here are the unique characteristics of each data set:

| | Offense-Column        | Offense-Value          | Reported-Column  | Reported-Data Type |
|-|-----------------------|------------------------|-----------------------------------|
| New York | `offenseDescription`  | starts with "murder" or "homicide" | `reportDate`     | `timestamp`    |
| Boston | `OFFENSE_CODE_GROUP`  | "Homicide"             | `MONTH`          | `integer`      |

In [9]:
from pyspark.sql.functions import lower, upper, month, col

crimeDataNewYorkDF = spark.read.parquet("/mnt/training/crime-data-2016/Crime-Data-New-York-2016.parquet")
crimeDataBostonDF = spark.read.parquet("/mnt/training/crime-data-2016/Crime-Data-Boston-2016.parquet")

homicidesNewYorkDF = (crimeDataNewYorkDF 
  .select(month(col("reportDate")).alias("month"), col("offenseDescription").alias("offense")) 
  .filter(lower(col("offenseDescription")).contains("murder") | lower(col("offenseDescription")).contains("homicide"))
)

homicidesBostonDF = (crimeDataBostonDF 
  .select("month", col("OFFENSE_CODE_GROUP").alias("offense")) 
  .filter(lower(col("OFFENSE_CODE_GROUP")).contains("homicide"))
)

# Join both DataFrames into a new one
homicidesBostonAndNewYorkDF = homicidesNewYorkDF.union(homicidesBostonDF)

Let's take a look at the new DataFrame to get a sense of the data we will be writing to Cosmos DB.

In [11]:
display(homicidesBostonAndNewYorkDF)

month,offense
12,MURDER & NON-NEGL. MANSLAUGHTER
12,MURDER & NON-NEGL. MANSLAUGHTER
12,MURDER & NON-NEGL. MANSLAUGHTER
12,MURDER & NON-NEGL. MANSLAUGHTER
12,MURDER & NON-NEGL. MANSLAUGHTER
12,MURDER & NON-NEGL. MANSLAUGHTER
12,MURDER & NON-NEGL. MANSLAUGHTER
12,MURDER & NON-NEGL. MANSLAUGHTER
12,MURDER & NON-NEGL. MANSLAUGHTER
12,MURDER & NON-NEGL. MANSLAUGHTER


## Write data to Cosmos DB

You write data back to Cosmos DB by creating a DataFrame that contains the desired changes and then use the Write API to save the changes back. 

As Spark DataFrames are immutable, if you want to insert new rows to a collection in Cosmos DB, you will need to create a new DataFrame. If you want to both modify and insert, the connector supports the upsert operation.

The Write API requires similar configuration to the Read API we used previously. Take note of the Upsert parameter we use here, which enables Upsert actions on the Cosmos DB side.

In [15]:
writeConfig = {
"Endpoint" : uri,
"Masterkey" : key,
"Database" : "demos",
"Collection" : "documents",
"Upsert" : "true"
}

By default, the Connector will not allow us to write to a collection that has documents already. In order to perform an upsert, we must specify that mode is set to `overwrite`.

In [17]:
homicidesBostonAndNewYorkDF.write.format("com.microsoft.azure.cosmosdb.spark").mode("overwrite").options(**writeConfig).save()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-3134093487317039> in <module> () 
 ----> 1 homicidesBostonAndNewYorkDF . write . format ( "com.microsoft.azure.cosmosdb.spark" ) . mode ( "overwrite" ) . options ( ** writeConfig ) . save ( ) 

 /databricks/spark/python/pyspark/sql/readwriter.py in save (self, path, format, mode, partitionBy, **options) 
 730 self . format ( format ) 
 731 if path is None : 
 --> 732 self . _jwrite . save ( ) 
 733 else : 
 734 self . _jwrite . save ( path ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o1521.save.
: java.lang.IllegalStateException: Cannot find Collection's corresponding offer
	at com.microsoft.azure.cosmosdb.spark.CosmosDBConnection.getCollectionThroughput(CosmosDBConnection.scala:143)
	at com.microsoft.azure.cosmosdb.spark.CosmosDBSpark$.save(CosmosDBSpark.scala:173)
	at com.microsoft.azure.cosmosdb.spark.CosmosDBSpark$.save(CosmosDBSpark.scala:501)
	at com.microsoft.azure.cosmosdb.spark.DefaultSource.createRelation(DefaultSource.scala:74)
	at org.apache.spark.sql.execution.datasources.SaveIntoDataSourceCommand.run(SaveIntoDataSourceCommand.scala:45)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:72)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult(commands.scala:70)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.doExecute(commands.scala:88)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:146)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:134)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$5.apply(SparkPlan.scala:187)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:183)
	at org.apache.spark.sql.execution.SparkPlan.execute(SparkPlan.scala:134)
	at org.apache.spark.sql.execution.QueryExecution.toRdd$lzycompute(QueryExecution.scala:114)
	at org.apache.spark.sql.execution.QueryExecution.toRdd(QueryExecution.scala:114)
	at org.apache.spark.sql.DataFrameWriter$$anonfun$runCommand$1.apply(DataFrameWriter.scala:710)
	at org.apache.spark.sql.DataFrameWriter$$anonfun$runCommand$1.apply(DataFrameWriter.scala:710)
	at org.apache.spark.sql.execution.SQLExecution$$anonfun$withCustomExecutionEnv$1.apply(SQLExecution.scala:111)
	at org.apache.spark.sql.execution.SQLExecution$.withSQLConfPropagated(SQLExecution.scala:240)
	at org.apache.spark.sql.execution.SQLExecution$.withCustomExecutionEnv(SQLExecution.scala:97)
	at org.apache.spark.sql.execution.SQLExecution$.withNewExecutionId(SQLExecution.scala:170)
	at org.apache.spark.sql.DataFrameWriter.runCommand(DataFrameWriter.scala:710)
	at org.apache.spark.sql.DataFrameWriter.saveToV1Source(DataFrameWriter.scala:306)
	at org.apache.spark.sql.DataFrameWriter.save(DataFrameWriter.scala:292)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl

## Challenge

Read the data back out of Cosmos DB, using the Read Connector you configured earlier.

In [20]:
# TODO
documents = spark.read.format("com.microsoft.azure.cosmosdb.spark").options(**readConfig).load()
# Add your code here, saving to a DataFrame named 'documents'

Display the contents of the `documents` DataFrame to ensure the data was successfully retrieved.

In [22]:
display(documents)

## Reference

If you wish to apply what you have learned so far to a new set of challenges, complete the optional exercise: [Exploratory Data Analysis]($./Optional/Exploratory-Data-Analysis)